In [1]:
import os
import sys
sys.path.append('/home/g/Desktop/git/nodules/src/')
sys.path.append('/home/g/Desktop/git/nodules/keras/')
sys.path.append('/home/g/Desktop/git/nodules/tf/')

import init
import kerasUtil as ku
import tan
import segutil as seg

import math
import scipy
import skimage.feature
import scipy.ndimage
import scipy.misc as scipy_misc
import scipy.ndimage.morphology as morphology
import numpy as np

import matplotlib.pyplot as plt

from skimage import measure

Using TensorFlow backend.


In [2]:
cfg = ['/scr/nodules/no4/val/bin4/','/scr/nodules/no4/samples/val/']
images, labels = init.loadpng(cfg)
#images, labels = init.loadData(cfg)
images, labels = ku.cleanData(images,labels)
testImages = ku.to3d(images,2)
testLabels = ku.get3Labels(labels)

In [3]:
D1 = 4.
D2 = 16.
no = 6
def getScales(opts,D1=4.,D2=16.,no=6):
  d0, d1, no = D1, D2, no
  sigma = []
  sigma.append(d0/4.)
  for i in range(1, no):
    val = ((d1/d0)**(1./(no-1.)))**i*sigma[0]
    sigma.append(val)
  print sigma
  return sigma

In [4]:
def Hessian(L):
  hessian = np.zeros((L.ndim,L.ndim)+L.shape,dtype=L.dtype)
  dL = np.gradient(L)
  for i, d1 in enumerate(dL):
    d2L = np.gradient(d1)
    for j, d2 in enumerate(d2L):
      hessian[i,j] = d2
  return hessian

In [5]:
def multiscaleHessian2d(image, opts):
  scales = getScales(opts)
  cands = []
  for Sigma in scales:
    img = np.zeros(image.shape)
    hess = np.zeros((2,2)+image.shape)
    sig = np.zeros(image.shape)
    for i in range(image.shape[0]):
      img[i,:,:] = scipy.ndimage.filters.gaussian_filter(image[i,:,:],sigma=Sigma,order=0) 
      hess[:,:,i,:,:] = Hessian(img[i,:,:])
    cands.append(hess)
  return cands

In [6]:
THRESH = 1.0
def filterHessian(hessians, opts):
  hess = hessians[0]
  scales = getScales(opts)
  cands = np.zeros((hess.shape[2],hess.shape[3],hess.shape[4]))
  candidates = np.zeros((hess.shape[2],hess.shape[3],hess.shape[4]))
  lambdas = np.zeros((hess.shape[2],hess.shape[3],hess.shape[4]))
  for h in range(len(hessians)):
    hessian = hessians[h]
    for i in range(hess.shape[2]):
      for j in range(hess.shape[3]):
        for k in range(hess.shape[4]):
          lambdas = np.linalg.eigvals(hessian[:,:,i,j,k])
          l1 = lambdas[0]
          l2 = lambdas[1]
          if abs(l2) > abs(l1):
            tmp = l1
            l1 = l2
            l2 = tmp
          if l1 < 0 and l2 < 0:
            response = (l2*l2)/abs(l1)
            if response > THRESH:
              print "RESPONSE " + str(response) + '  L1 ' + str(l1) + '  L2 ' + str(l2)
            if response > candidates[i,j,k] and response > THRESH:
              candidates[i,j,k] = 1
  return candidates

In [7]:
img = testImages[3]
opts = None
cands = multiscaleHessian2d(img, opts)
newCands = filterHessian(cands, opts)

[1.0, 1.3195079107728942, 1.741101126592248, 2.2973967099940698, 3.0314331330207955, 3.999999999999999]
[1.0, 1.3195079107728942, 1.741101126592248, 2.2973967099940698, 3.0314331330207955, 3.999999999999999]
RESPONSE 3.53883873371  L1 -10.2619793581  L2 -6.02623348675
RESPONSE 1.06074332335  L1 -2.01762845506  L2 -1.46293742611
RESPONSE 2.03038123322  L1 -4.90487455243  L2 -3.15575113761
RESPONSE 4.19119665797  L1 -5.40015301441  L2 -4.75742611782
RESPONSE 2.51945360135  L1 -5.47031954806  L2 -3.7124407451
RESPONSE 6.1721158893  L1 -8.44945516909  L2 -7.22156607012
RESPONSE 8.7673669912  L1 -12.4318666572  L2 -10.4400544716
RESPONSE 6.65613839634  L1 -8.75042749673  L2 -7.6317793761
RESPONSE 2.11500713919  L1 -6.28703645596  L2 -3.64652258854
RESPONSE 3.56471346247  L1 -6.73149206269  L2 -4.89855492757
RESPONSE 2.31423572584  L1 -6.01423566149  L2 -3.73073170189
RESPONSE 4.06645473282  L1 -10.7533322713  L2 -6.61271040558
RESPONSE 1.85478718855  L1 -8.71286794342  L2 -4.02001440756
RES

In [ ]:
from skimage import feature
roi = img
scales = getScales(opts)
eigenvalues = []
hessians = []
for Sigma in scales:
  l1 = np.zeros(roi.shape)
  l2 = np.zeros(roi.shape)
  hessian_matrix = np.zeros((2,2)+roi.shape)
  for i in range(roi.shape[0]):
    hxx, hxy, hyy = feature.hessian_matrix(roi[i,:,:],sigma=Sigma)
    hessian_matrix[0,0] = hxx
    hessian_matrix[0,1] = hxy
    hessian_matrix[1,0] = hxy
    hessian_matrix[1,1] = hyy
    l1[i,:,:], l2[i,:,:] = feature.hessian_matrix_eigvals(hxx,hxy,hyy)
  eigenvalues.append((l1,l2))
  hessians.append(hessian_matrix)
THRESH = 0.00
candidates = np.zeros(roi.shape)
for lambdas in eigenvalues:
  lambda1, lambda2 = lambdas
  for i in range(roi.shape[0]):
    for j in range(roi.shape[1]):
      for k in range(roi.shape[2]):
        l1 = lambda1[i,j,k]
        l2 = lambda2[i,j,k]
        lambdasAlt = np.linalg.eigvals(hessian_matrix[:,:,i,j,k])
        l1 = lambdasAlt[0]
        l2 = lambdasAlt[1]
        if l1 < 0 and l2 < 0:
          if l1 > l2:
            response = (l1*l1)/abs(l2)
          else:
            response = (l2*l2)/abs(l1)
          if response > THRESH:
            print "RESPONSE " + str(response) + '  L1 ' + str(l1) + '  L2 ' + str(l2)
          if response > candidates[i,j,k] and response > THRESH:
            candidates[i,j,k] = 1

[1.0, 1.3195079107728942, 1.741101126592248, 2.2973967099940698, 3.0314331330207955, 3.999999999999999]
RESPONSE 0.00968844617557  L1 -1.17198285566  L2 -0.106558400963
RESPONSE 0.035232614903  L1 -0.93341033425  L2 -0.181346317451
RESPONSE 0.0610737911727  L1 -0.715709105063  L2 -0.209071921651
RESPONSE 0.0594671001182  L1 -0.182146614787  L2 -0.557911672377
RESPONSE 0.0228656499495  L1 -0.104293739708  L2 -0.475699757772
RESPONSE 0.000355044414507  L1 -0.0124995310565  L2 -0.440052765932
RESPONSE 0.0197652105269  L1 -0.0944384215617  L2 -0.451227952009
RESPONSE 0.119191931588  L1 -0.233995123274  L2 -0.459374363574
RESPONSE 0.311575592082  L1 -0.377989096323  L2 -0.458558887699
RESPONSE 0.357669654015  L1 -0.428874662586  L2 -0.51425519091
RESPONSE 0.245554079242  L1 -0.393550597457  L2 -0.6307452649
RESPONSE 0.157900741945  L1 -0.337534501752  L2 -0.721526311209
RESPONSE 0.0939679537545  L1 -0.272737656212  L2 -0.791608480803
RESPONSE 0.051646912046  L1 -0.210446186063  L2 -0.857507

In [ ]:
newCands = candidates

In [ ]:
for i in range(16):
  print i
  plt.imshow(newCands[i,:,:])
  plt.show()
  plt.clf()
  plt.imshow(img[i,:,:])
  plt.show()
  plt.clf()

In [ ]:
VOLUME_THRESH = 15

labels = np.zeros(newCands.shape)
labels = measure.label(newCands,connectivity=1)
cands = np.zeros(newCands.shape)
mx = np.max(labels)
for i in range(1,int(mx)):
  count = np.sum(newCands[labels == i])
  #print str(i) + ' ' + str(count)
  if count > VOLUME_THRESH:
    cands[labels == i] = i
  #print str(i) + ' ' + str(np.sum(cands[labels == i])/i)
#print cands
moreCands = np.zeros(cands.shape)
moreCands[cands > 0] = 1

In [ ]:
cands[:,0,0] = 100.
for i in range(16):
  print i
  plt.imshow(moreCands[i,:,:])
  plt.show()
  plt.clf()
  plt.imshow(img[i,:,:])
  plt.show()
  plt.clf()

In [ ]:
labels = measure.label(moreCands)
#for i in range(0,int(mx)):
#  print str(i) + ' ' + str(np.sum(moreCands[labels == i]))

In [ ]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot3d(image, threshold=-300):
  img = image.transpose(2,1,0)
  vertices, faces = measure.marching_cubes(img, threshold)
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(111, projection='3d')
  mesh = Poly3DCollection(vertices[faces], alpha=0.70)
  faceColor = [0.45, 0.45, 0.75]
  mesh.set_facecolor(faceColor)
  ax.add_collection3d(mesh)
  ax.set_xlim(0, img.shape[0])
  ax.set_ylim(0, img.shape[1])
  ax.set_zlim(0, img.shape[2])
  plt.show()

In [ ]:
plot3d(cands,1)